In [1]:
#pip install importnb

In [2]:
# This notebook executes the training routines of the agent(s) 
import random
from importnb import Notebook
with Notebook():
    #import Board
    import Agent
from abc import ABC, abstractmethod

Columna inválida. Elija un número entre 1 y 7.
Columna inválida. Elija un número entre 1 y 7.
Ficha no valida
Columna llena. Elija otra columna.
All tests passed successfully!


In [3]:
#### Parent Class ####

In [4]:
class Game(ABC):
    def __init__(self, board, agent1):
        """
        clase abstracta Game
        """
        self.board = board
        self.agent1 = agent1

    def reset_game(self):
        self.board.reset()

    @abstractmethod
    def play(self):
        pass # To be implemented by the child classes

In [5]:
#### Child Classes ####

In [6]:
class AgentVsAgent(Game):
    def __init__(self, board, agent1, agent2, epochs = 1):
        super().__init__(board, agent1)
        self.epochs = epochs
        self.agent2 = agent2

    def play(self):
        """Implementa los juegos entre los dos agentes."""
        for epoch in range(self.epochs):
            self.reset_game()
            player_turn = random.choice([True, False]) # randomize who gets first turn
            
            while not self.board.is_game_over():
                if player_turn:
                    # TODO: implementar el juego del agente1 -> usar la misma de la clase de abajo EXCEPTO por el epsilon decay
                    pass
                else:
                    # TODO: implementar el juego del agente2 -> usar la misma de la clase de abajo EXCEPTO por el epsilon decay
                    pass
                player_turn = not player_turn

In [17]:
class PlayerVsAgent(Game):
    def __init__(self, board, agent):
        super().__init__(board, agent)
        self.agent = agent
        #self.player_chip = ?
        if self.agent.chip == "x":
            self.player_chip = "o"
        else:
            self.player_chip = "x"

    def play(self):
        """Implementa un juego entre un humano y un agente."""
                
        play_again = True
        while play_again:
            
            self.reset_game()
            player_turn = random.choice([True, False]) # randomize who gets first turn
            
            while not self.board.is_game_over():              
                
                if player_turn:
                    # print the current state of the board
                    self.board.print_board()
                    print("It´s your turn!")

                    # Ask for the next move and place the chip
                    while True:
                        try:
                            user_input = input("What is your move? ")
                            new_col_number = int(user_input)  # Intentar convertir a entero
                            
                            # Verificar que el número esté dentro del rango permitido
                            if not (1 <= new_col_number <= 7):
                                raise ValueError("The column must be a number between 1 and 7.")
                            
                            # Intentar colocar la ficha en el tablero
                            if not self.board.place_chip(new_col_number, self.player_chip):
                                raise ValueError("Invalid move. The column might be full or invalid. Try again.")
                            
                            # Si el movimiento es válido, romper el bucle
                            break
                            
                        except ValueError as e:
                            print(e)

                    # Check if game is over
                    if self.board.verify_winner(self.player_chip):
                        print("YOU WIN!")
                        # comunicarle al agente que perdió
                        break
                    elif self.board.isBoardFull():
                        print("ITS A TIE")
                        # comunicarle al agente que empató
                        break
                    
                else:
                    print("It´s my turn!")

                    # Agente selecciona una jugada new_col_number = agent.choose_action(agent.get_state()) o algo asi
                    # Poner la ficha con self.board.place_chip(new_col_number, self.agent_chip):
                                        # Check if game is over
                    if self.board.verify_winner(self.agent.chip):
                        print("YOU LOSE!")
                        # TODO: Recompensas
                        break
                    elif self.board.isBoardFull():
                        print("ITS A TIE")
                        # TODO: Recompensas
                        break
                    # Asignar recompensa y actualizar Q-Tabla

                player_turn = not player_turn

            # Juego terminó, preguntar si se quiere volver a jugar
            usr_input = ""
            while usr_input.lower() not in ["y", "n", "yes", "no"]:
                usr_input = input("Do you want to play again? (y/n): ").strip()

            if usr_input.lower() in ["y", "yes"]:
                play_again = True  # No es necesario redefinir, pero preferi dejarlo explícito
            elif usr_input.lower() in ["n", "no"]:
                play_again = False

        print("Thanks for playing, Bye!")

In [8]:
### Aux Functions ###

def chooseChips():
        # Choose the chips (player can choose)
    player1_chip = ""
    while player1_chip.lower() not in ["x", "o"]:
        player1_chip = input("Please select your chip (x or o)").strip()
        
    player1_chip = player1_chip.lower()        
    if player1_chip == "x":
        player2_chip = "o"
    else:
        player2_chip = "x"
        
    return player1_chip, player2_chip

def createAgent(): # hay que primero definir como se va a construir el agente para definir esta funcion
    # player1_chip, player2_chip = chooseChips():
    #return Agent(...,player2_chip,...)
    pass
